<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
<img src="http://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/UOC_Masterbrand.jpg", align="left">
</div>
<div style="float: right; width: 50%;">
<p style="margin: 0; padding-top: 22px; text-align:right;">M2.852 · Tipología y ciclo de vida de los datos · PRA1</p>
<p style="margin: 0; text-align:right;">2022-2 · Máster universitario en Ciencia de datos (Data science)</p>
<p style="margin: 0; text-align:right; padding-button: 100px;">Estudios de Informática, Multimedia y Telecomunicación</p>
</div>
</div>
<div style="width:100%;">&nbsp;</div>


# PRA1: Web scraping - Testing file

In [1]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install builtwith

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install python-whois

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install scrapy

Note: you may need to restart the kernel to use updated packages.


In [6]:
import requests
import builtwith
import whois
from bs4 import BeautifulSoup
import re
import scrapy
import pandas as pd

In [7]:
# Let's check the technologies of the webpage we want to scrap
builtwith.parse('https://www.expatistan.com/cost-of-living/country/ranking')

{'web-servers': ['Nginx'], 'advertising-networks': ['Google AdSense']}

In [8]:
# Let's see the content of the webpage
page = requests.get("https://www.expatistan.com/cost-of-living/country/ranking")
page.content

b'<!DOCTYPE html>\n<html lang="en">\n<head>\n  <meta http-equiv="content-type" content="text/html; charset=utf-8" />\n                  <link rel="alternate" hreflang="en" href="https://www.expatistan.com/cost-of-living/country/ranking" />\n                <link rel="alternate" hreflang="es" href="https://www.expatistan.com/es/costo-de-vida/pais/ranking" />\n                <link rel="alternate" hreflang="pt" href="https://www.expatistan.com/pt/custo-de-vida/pais/classificacao" />\n          <title>Cost of Living Ranking by countries. Updated Apr 2023</title>\n<meta name="description" content="Ranking of the the cheapest and the most expensives countries in the world with current prices as of Apr 2023." />\n<meta property="og:title" content="Cost of Living Ranking by countries. Updated Apr 2023"/>\n<meta property="og:type" content="website"/>\n<meta property="og:description" content="Ranking of the the cheapest and the most expensives countries in the world with current prices as of Ap

In [9]:
# Let's see the owner of the domain
print(whois.whois("expatistan.com"))

{
  "domain_name": [
    "EXPATISTAN.COM",
    "expatistan.com"
  ],
  "registrar": "NAMECHEAP INC",
  "whois_server": "whois.namecheap.com",
  "referral_url": null,
  "updated_date": [
    "2018-07-27 13:28:22",
    "2018-07-27 13:28:22.640000"
  ],
  "creation_date": "2009-08-16 09:25:31",
  "expiration_date": "2023-08-16 09:25:31",
  "name_servers": [
    "NS-1210.AWSDNS-23.ORG",
    "NS-1673.AWSDNS-17.CO.UK",
    "NS-383.AWSDNS-47.COM",
    "NS-667.AWSDNS-19.NET",
    "NS1.LINODE.COM",
    "NS2.LINODE.COM",
    "NS3.LINODE.COM",
    "NS4.LINODE.COM",
    "NS5.LINODE.COM",
    "ns1.linode.com",
    "ns2.linode.com",
    "ns3.linode.com",
    "ns4.linode.com",
    "ns5.linode.com",
    "ns-667.awsdns-19.net",
    "ns-383.awsdns-47.com",
    "ns-1673.awsdns-17.co.uk",
    "ns-1210.awsdns-23.org"
  ],
  "status": "clientTransferProhibited https://icann.org/epp#clientTransferProhibited",
  "emails": [
    "abuse@namecheap.com",
    "7bf6de4f898b43a68152bbb9aed7ec79.protect@withheldforpr

In [10]:
# Let's analyse the structure of the html code
soup = BeautifulSoup(page.content)
print(soup.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="content-type"/>
  <link href="https://www.expatistan.com/cost-of-living/country/ranking" hreflang="en" rel="alternate"/>
  <link href="https://www.expatistan.com/es/costo-de-vida/pais/ranking" hreflang="es" rel="alternate"/>
  <link href="https://www.expatistan.com/pt/custo-de-vida/pais/classificacao" hreflang="pt" rel="alternate"/>
  <title>
   Cost of Living Ranking by countries. Updated Apr 2023
  </title>
  <meta content="Ranking of the the cheapest and the most expensives countries in the world with current prices as of Apr 2023." name="description"/>
  <meta content="Cost of Living Ranking by countries. Updated Apr 2023" property="og:title"/>
  <meta content="website" property="og:type"/>
  <meta content="Ranking of the the cheapest and the most expensives countries in the world with current prices as of Apr 2023." property="og:description"/>
  <meta content="//d2y05869ftj0yg.cloudfron

# Web scraping the URLS

In [11]:
class ExpatistanSpider(scrapy.Spider):
    
    def __init__(self):
        self.original_domain = "https://www.expatistan.com/cost-of-living/country/ranking"
        self.dataset =  {
            "Ranking position": [],
            'Country': [],
            'Category': [],
            'Items': [],
            'Original Currency': [],
            'Original Currency Value': [],
            'Exchanged Currency': [],
            'Exchanged Currency Value': []
        }
         
    def __get_HTML(self, domain):
        page = requests.get(domain)
        return BeautifulSoup(page.content)
        
    def __get_countries_links(self, html):
        """
        Returns a list of the links to be scraped

        Parameters
        ----------
        self: Instance of the class that is invoking the method
        html: HTML code of the original page

        Returns
        -------
        List of string values

        """
        tds = html.find_all('td', {"class": "country-name"})
        countries_links = [td.find('a').get('href') + "?currency=EUR" for td in tds]
        return countries_links
    
    def __get_ranking_pos(self, html):
        li_labels = html.findAll('li', {'class': 'key-point'})
        for li in li_labels:
            if 'World' in li.text:
                text = li.text.split("World", 1)[1].strip()
                pos = re.search(r'\b(\d{1,2})\b', text).group(1)
        return pos

    def scraping(self):
        # 1. Get the original HTML file using Beautiful Soup
        html = self.__get_HTML(self.original_domain)
        
        # 2. Get all the domains to scrape 
        country_domains = self.__get_countries_links(html)
        
        # 3. For each country, let's scrape all the information
        for domain in country_domains:
            country = re.search(r"country/([^/?]*)", domain).group(1)
            country = re.sub("-", " ", country).title()
            print("Scraping country " + country)
            self.__scraping_single_country(domain, country)

        self.__saving()
            
        
    def __scraping_single_country(self, domain, country):
        # First we need to get the HTML file for this domain
        domain_html = self.__get_HTML(domain)
        
        # Then we get the Ranking position
        pos = self.__get_ranking_pos(domain_html)

        # All the information is under <tr> labels, let's find them
        tr_labels = domain_html.findAll('tr')

        # But not all of them have value to the proyect, so we need to remove them
        tr_labels.pop(-1)

        for i, tr in enumerate(tr_labels):
            # Getting the first td or th label of the tr label
            td = tr.find("td")
            th = tr.find("th")

            # Check if its one of the non-valuable types of label
            if td and td.get("colspan") == "4":
                # If so, we get rid of it
                tr_labels.pop(i)
            elif th and th.get("class") == ["ranking"]:
                # If so, we get rid of it
                tr_labels.pop(i)

        # Setting the common variables for teh scraping
        current_category = ""
        current_orig_currency = ""

        for tr in tr_labels:
            if "categoryHeader" in tr.get("class", []):
                texto = tr.find("th")
                current_category = texto.text
            else:
                td_labels = tr.findAll("td")

                if len(td_labels) == 2:
                    currency = tr.find("td")
                    current_orig_currency = currency.text

                elif len(td_labels) == 3:            
                    self.dataset['Ranking position'].append(pos)
                    self.dataset['Country'].append(country)
                    self.dataset['Category'].append(current_category)
                    self.dataset['Items'].append(td_labels[1].text.strip())
                    self.dataset['Original Currency'].append("EUR")
                    self.dataset['Original Currency Value'].append((td_labels[2].text.strip()))
                    self.dataset['Exchanged Currency'].append("EUR")
                    self.dataset['Exchanged Currency Value'].append((td_labels[2].text.strip()))
                

                elif len(td_labels) == 4: 
                    self.dataset['Ranking position'].append(pos)
                    self.dataset['Country'].append(country)
                    self.dataset['Category'].append(current_category)
                    self.dataset['Items'].append(td_labels[1].text.strip())
                    self.dataset['Original Currency'].append(current_orig_currency)
                    self.dataset['Original Currency Value'].append((td_labels[2].text.strip()))
                    self.dataset['Exchanged Currency'].append("EUR")
                    self.dataset['Exchanged Currency Value'].append((td_labels[3].text.strip()))
                    
            
    def __saving(self):
        # Let's create a Pandas Dataframe with the obtained dataset
        expatistan_df = pd.DataFrame(self.dataset)
        
        # Now we save it as a CSV
        expatistan_df.to_csv("expatistan.csv", index = False)


In [12]:
scraper = ExpatistanSpider()
scraper.scraping()

Scraping country Bermuda
Scraping country Singapore
Scraping country Switzerland
Scraping country Cayman Islands
Scraping country Bahamas
Scraping country Hong Kong
Scraping country Ireland
Scraping country United States
Scraping country Denmark
Scraping country Norway
Scraping country Netherlands
Scraping country Luxembourg
Scraping country Australia
Scraping country Israel
Scraping country New Zealand
Scraping country Qatar
Scraping country United Kingdom
Scraping country Canada
Scraping country Germany
Scraping country Finland
Scraping country Belgium
Scraping country Japan
Scraping country Sweden
Scraping country United Arab Emirates
Scraping country France
Scraping country Austria
Scraping country Macao
Scraping country Malta
Scraping country South Korea
Scraping country Uruguay
Scraping country Italy
Scraping country Panama
Scraping country Palestinian Territory
Scraping country Dominican Republic
Scraping country Kuwait
Scraping country Costa Rica
Scraping country Bahrain
Scrapi